## 現在のファイル状態の確認

In [ ]:
# import pandas as pd
# import os
# from mymodule import save_bin,read_bin,directory_to_dict

# Xeno2sound_ids = read_bin("saved_data/Xeno2sound_ids_all.bin")

# directory_path = 'E:\\xeno_data'
# result = directory_to_dict(directory_path)

In [ ]:
# df = pd.read_csv("saved_data/id2Xeno.csv",header = None)# 順番通りに取得する必要があるため
# the_ids = list(df[0])[1:]
# xeno_canto_id = list(df[1])[1:]

In [ ]:
# Wiki2Xeno = dict()
# for k,v in zip(the_ids,xeno_canto_id):
#     Wiki2Xeno[k] = v

In [ ]:
# # #ファイルが一つもないディレクトリを確認
# nodata_files = []
# for the_id in the_ids:
#     if the_id in result and result[the_id] == []:
#         nodata_files.append(the_id)

# #ファイルが欠損したディレクトリ（101未満かつ本来のファイル数に満たない）
# lack_files = []
# lack_files_num = []
# for the_id in the_ids:
#     if the_id in result and the_id in Xeno2sound_ids:
#         if result[the_id] != []:
#             if len(result[the_id])>=101:
#                 pass
#             else:
#                 d = len(Xeno2sound_ids[the_id])-len(result[the_id])
#                 if d!=0:
#                     lack_files.append(the_id)
#                     lack_files_num.append(d)

# # そもそも存在しないディレクトリ
# not_in_Xeno2sound_ids_files = []
# for the_id in the_ids:
#     if the_id in result:
#         if the_id not in Xeno2sound_ids:
#             not_in_Xeno2sound_ids_files.append(the_id)


# print("nodata files:",len(nodata_files))
# print("lack_files:",len(lack_files))
# print("not_in_Xeno2sound_ids_files:",len(not_in_Xeno2sound_ids_files))

nodata files: 378
lack_files: 0
not_in_Xeno2sound_ids_files: 0


## 学習用parquetデータの作成

In [2]:
from mymodule import save_bin,read_bin,directory_to_dict
import os

# ディレクトリ=>辞書=>タプル
# result = directory_to_dict('E:\\xeno_data')
# xeno_data_tuple = set()
# for bird_id,sound_ids in result.items():
#     for sound_id in sound_ids:
#         xeno_data_tuple.add((bird_id,sound_id))

# save_bin("./saved_data/xeno_data_tuple.bin",xeno_data_tuple)
# xeno_data_tuple = read_bin("./saved_data/xeno_data_tuple.bin")#337644

In [24]:
# import importlib
from mymodule3 import save_bin,read_bin,directory_to_dict,directory_to_tuple
import os

trimmed_xeno_data_tuple = directory_to_tuple("D:\\trimmed_xeno_data")
dowmload_xeno_data_tuple = directory_to_tuple("D:\\download_xeno_data") 

print(len(trimmed_xeno_data_tuple),"/",len(dowmload_xeno_data_tuple))

403230 / 0


In [25]:
target_set = list(dowmload_xeno_data_tuple - trimmed_xeno_data_tuple)
target_set = list(target_set)
print(len(target_set))

# save_bin("saved_data/target_set.bin",target_set)

0


## 無音域削除，ファイル長統一

In [24]:
# from mymodule2 import save_bin,read_bin,directory_to_dict,process_audio
# n = 1

# target_set = read_bin("saved_data/target_set.bin")

# x = 3000*(n-1)
# y = x + 3000

# target_set = target_set[x:y]
# print(x,y)

# i=0
# for input_tuple in target_set:
#     print('\r%d / %d' %(i, len(target_set)), end='')
#     process_audio(input_tuple)
#     i+=1

In [4]:
import os
from concurrent.futures import ProcessPoolExecutor, as_completed
from pydub import AudioSegment
from pydub.silence import split_on_silence


# path = "E:\\xeno_data\\"
# path_trimmed = "E:\\trimmed_xeno_data\\"

path = ".\\xeno_data"
path_trimmed = ".\\trimmed_xeno_data"


def process_audio(input_tuple,path,path_trimmed):
    try:
        bird_id, sound_id = input_tuple
        if os.path.exists(path+"\\"+bird_id):
            sound = AudioSegment.from_mp3(path+"\\"+bird_id+"\\"+sound_id)
            chunks = split_on_silence(
                sound,
                min_silence_len=500,  # 0.5秒以上の無音を検出
                silence_thresh=-90    # -90dBFS 以下を無音と判定
            )

            combined = AudioSegment.empty()
            for chunk in chunks:
                combined += chunk

            if len(combined) > 30_000:
                combined = combined[:30_000]
            
            if not os.path.exists(path_trimmed+"\\"+bird_id):
                os.makedirs(path_trimmed+"\\"+bird_id)
                
            combined.export(path_trimmed+"\\"+bird_id+"\\"+sound_id, format="mp3")

    except:
        return input_tuple

In [2]:
# from mymodule import save_bin,read_bin,directory_to_dict,directory_to_tuple
# import os
# from sklearn.model_selection import KFold

# result = list(directory_to_tuple("D:\\trimmed_xeno_data"))
# kf = KFold(n_splits=5, shuffle=True, random_state=42)
# kfold_5_test = []

# for _, (_, test_index) in enumerate(kf.split(result), start=1):
#     kfold_5_test.append([result[i] for i in test_index])

In [7]:
del results

In [1]:
from mymodule import save_bin,read_bin,directory_to_dict,directory_to_tuple
# save_bin("saved_data/kfold_5_test.bin",results)
# kfold_5_test = read_bin("saved_data/kfold_5_test.bin")

# kfold_5_train = []
# for i in range(5):
#     train = []
#     for j in range(5):
#         if i != j:
#             train.extend(kfold_5_test[j])

#     kfold_5_train.append(train) 

# save_bin("saved_data/kfold_5_train.bin",kfold_5_train)
kfold_5_train = read_bin("saved_data/kfold_5_train.bin")

In [ ]:
# path情報のみのparquetファイルの作成
import os
import pandas as pd
from pydub import AudioSegment

for x,train in enumerate(kfold_5_train):
    data = []
    i = 1
    for bird_id,sound_id in train:
        print('\r%d : %d / %d' %(x, i, len(train)), end='')
        file_path = os.path.join("trimmed_xeno_data",bird_id, sound_id)
        file_path_linux = file_path.replace("\\","/")
        data.append([sound_id.replace(".mp3",""),file_path_linux,bird_id])
        i = i + 1
    data_df = pd.DataFrame(data, columns=["s_id","audio","description"])
    data_df.set_index("s_id", inplace=True)
    data_df.index.name = None
    data_df.to_parquet('E:\\wav_model_data\\'+str(x+1)+'_train.parquet')

In [24]:
df = pd.read_parquet('E:\\wav_model_data\\5_train.parquet')

In [25]:
df

,audio,description


In [14]:
import pandas as pd
import glob

file_list = sorted(glob.glob("E:\\wav_model_data\\*.parquet"))
for i in range(1, 6):
    test_file = f"'E:\\wav_model_data\\{i}.parquet"
    train_files = [f for f in file_list if f != test_file]
    train_df = pd.concat([pd.read_parquet(f) for f in train_files])
    train_df.to_parquet('E:\\wav_model_data\\'+str(i)+'\\train.parquet', index=False)

In [15]:
for i in range(1, 6):
    test_file = f"'E:\\wav_model_data\\{i}.parquet"
    print(len(test_file))

28
28
28
28
28


In [1]:
# # origilal

# import os
# import pandas as pd
# from pydub import AudioSegment

# data = []
# error = []
# start = 0

# i = 0

# for bird_id,sound_ids in result[75]:
#     print('\r%d / %d' %(i, len(result)), end='')
#     for sound_id in sound_ids:
#         try:
#             file_path = os.path.join("xeno_data",bird_id, sound_id)
#             audio = AudioSegment.from_mp3(file_path)
#             wav_data = audio.export(format="wav").read()
#             wav_dict = {"bytes":wav_data,"path":None}
#             data.append([sound_id.replace(".mp3",""),wav_dict,bird_id])
#         except:
#             error.append(file_path)
#     if i % 100 == 0 or i == len(result)-1:
#         # DataFrameを作成
#         data_df = pd.DataFrame(data, columns=["s_id","audio","description"])
#         del data
#         data_df.set_index("s_id", inplace=True)
#         data_df.index.name = None
#         try:
#             data_df.to_parquet('E:\\wav_model_data\\all_xeno_~'+str(8900+i)+'.parquet')
#         except:
#             data_df.to_parquet('E\\wav_model_data\\all_xeno_~'+str(8900+i)+'.parquet')
#         # del data_df
#         data = []
#     i = i + 1